In [1]:
!pip install tensorflow==2.12 keras==2.12 larq

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.8 MB/s eta 0:00:00
   ━━━

In [1]:
import tensorflow as tf
import larq as lq
import numpy as np
import os

# Loading and preprocessing MNIST data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 784).astype("float32") / 127.5 - 1
test_images = test_images.reshape(-1, 784).astype("float32") / 127.5 - 1

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)


11490434/11490434 [==============================] - 0s 0us/step


In [2]:
# Defining binary quantization params
kwargs = dict(
    input_quantizer="ste_sign",
    kernel_quantizer="ste_sign",
    kernel_constraint="weight_clip",
    use_bias=False
)

# BNN model architecture according to the paper
model = tf.keras.models.Sequential([
    lq.layers.QuantDense(512, input_shape=(784,), **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(512, **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(512, **kwargs),
    tf.keras.layers.BatchNormalization(),

    lq.layers.QuantDense(10, **kwargs),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("softmax")
])


In [3]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_data=(test_images, test_labels))

print("\nOriginal Model Accuracy:")
orig_loss, orig_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {orig_acc * 100:.2f}%")

os.makedirs("model_trained", exist_ok=True)
model.save("model_trained/binarized_model", include_optimizer=False)

Epoch 1/20
469/469 [==============================] - 22s 41ms/step - loss: 0.5504 - accuracy: 0.8971 - val_loss: 0.4147 - val_accuracy: 0.9102
Epoch 2/20
469/469 [==============================] - 19s 40ms/step - loss: 0.2685 - accuracy: 0.9481 - val_loss: 0.2150 - val_accuracy: 0.9524
Epoch 3/20
469/469 [==============================] - 19s 40ms/step - loss: 0.1878 - accuracy: 0.9592 - val_loss: 0.1738 - val_accuracy: 0.9573
Epoch 4/20
469/469 [==============================] - 19s 41ms/step - loss: 0.1411 - accuracy: 0.9675 - val_loss: 0.1424 - val_accuracy: 0.9624
Epoch 5/20
469/469 [==============================] - 19s 41ms/step - loss: 0.1176 - accuracy: 0.9711 - val_loss: 0.1692 - val_accuracy: 0.9524
Epoch 6/20
469/469 [==============================] - 20s 42ms/step - loss: 0.1011 - accuracy: 0.9740 - val_loss: 0.1522 - val_accuracy: 0.9564
Epoch 7/20
469/469 [==============================] - 18s 39ms/step - loss: 0.0851 - accuracy: 0.9780 - val_loss: 0.1728 - val_accuracy: